In [3]:
pip install -U spacy -q


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import  tqdm

nlp=spacy.blank("en")
db=DocBin()

In [5]:
import json
f=open('./final_annotations.json')
TRAIN_DATA=json.load(f)

In [6]:

for ele in tqdm(TRAIN_DATA['annotations']):
    if ele is None:continue
    text=ele[0]
    annot=ele[1]
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 48/48 [00:00<00:00, 393.82it/s]


In [7]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    107.21    0.00    0.00    0.00    0.00
  5     200       2926.28   3217.29   10.78   22.45    7.10    0.11
 11     400       3448.91   1787.91   65.69   75.63   58.06    0.66
 17     600        194.93    766.91   72.09   65.61   80.00    0.72
 23     800        305.82    600.10   84.72   91.73   78.71    0.85
 29    1000        430.81    555.12   84.89   95.93   76.13    0.85
 36    1200        385.71    642.28   82.67   85.52   80.00    0.83
 46    1400        434.13    732.75   85.11   94.49   77.42    0.85
 58    1600      25096.26   1052.91 

In [ ]:
nlp_ner = spacy.load("/content/model-best")
doc=nlp_ner('''A PROPANE LEAK, IN THE PRESENCE OF AN IGNITION SOURCE, COULD RESULT IN A FIRE AND/OR EXPLOSION CAUSING PROPERTY DAMAGE AND/OR PERSONAL INJURY.	DEALERS WILL REPLACE THE PROPANE HOSE WITH ONE OF SHORTER LENGTH AND WILL ALSO SHORTEN THE HOSE SUPPORT TO MOVE THE HOSE AWAY FROM THE TIRE.  THE RECALL BEGAN ON FEBRUARY 1, 2008.  OWNERS MAY CONTACT KEYSTONE RV AT 1-866-425-4369.	KEYSTONE RV RECALL NO. 08-100.CUSTOMERS MAY ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATION'S VEHICLE SAFETY HOTLINE AT 1-888-327-4236 (TTY 1-800-424-9153), OR GO TO HTTP://WWW.SAFERCAR.GOV.
''')

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [ ]:
doc3=nlp_ner('''The incorrectly sized fuse may blow during an ABS braking event, increasing the risk of a crash.	Oshkosh will provide owners new fuses, a new fuse label, and installation instructions, free of charge.  Owner notification letters were mailed on July 29, 2021.  Owners may contact Oshkosh customer service at 1-888-686-7278.	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.nhtsa.gov.
''')
spacy.displacy.render(doc3, style="ent", jupyter=True)

*NER heavily rely on text data for contextual purpose since there is no context of vehicle model in training data it woule be able to predict vehicle name in many cases or in almost all cases apart from training data*

*The effectiveness of an NER model comes from understanding how entities relate to the context of the surrounding words. For example, "Ford" might be a vehicle model, but it could also appear as part of an organization name or in a different context. The model needs context to distinguish between these possibilities*

In [ ]:
import spacy
from spacy.training.example import Example

# Load your fine-tuned model
nlp = spacy.load('./model-best')
test_data=[
    'If the floor mat moves out of position and interferes with the accelerator or brake pedal operation, it can increase the risk of a crash.	Subaru has notified owners, and dealers will inspect the floor mat retention bracket and replace it if necessary, free of charge.  The recall began November 14, 2017.  Owners may contact Subaru customer service at 1-800-782-2783.  Subaru number for this recall is WTR-77.	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.safercar.gov.'
,'THE RIVETS MUST BE REPLACED WITH THE CORRECT BOLTS.	SYSTEM: INTERIOR SYSTEMS: SEAT TRACKS AND ANCHORS; REAR SEAT; FMVSS 207.VEHICLE DESCRIPTION: LIGHT-DUTY TRUCKS; 1-TON CREW CAB PICKUP TRUCKS EQUIPPEDWITH AM7 FOLDING REAR BENCH SEAT.CONSEQUENCE OF DEFECTS: IN AN ACCIDENT, THE SEAT WOULD BECOME DETACHEDCAUSING INJURIES TO THE SEAT OCCUPANTS OR OTHER OCCUPANTS IN THE TRUCK.NOTE: IF YOUR VEHICLE IS PRESENTED TO AN AUTHORIZED DEALER ON AN AGREED UPONSERVICE DATE AND THE REMEDY IS NOT PROVIDED WITHIN A REASONABLE TIME AND FREEOF CHARGE OR THE REMEDY DOES NOT CORRECT THE DEFECT OR NONCOMPLIANCE, PLEASECONTACT THE CHEVROLET SERVICE CENTER AT 1-800-222-1020 OR GMC TRUCK SERVICECENTER AT 1-313-456-4547. ALSO, CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFETYADMINISTRATION AUTO SAFETY HOTLINE AT 1-800-424-9393.'
,'THIS CONDITION CAN CONTINUE TO DETERIORATE AND WILL EVENTUALLY RESULT IN THE BALL JOINT STUD SEPARATING FROM THE HOUSING INCREASING THE RISK OF LOSS OF VEHICLE CONTROL.	MOOG WILL REPLACE THESE BALL JOINTS WITH BALL JOINTS FROM ANOTHER SUPPLIER.	OWNER NOTIFICATION BEGAN OCTOBER 12, 1998.OWNERS WHO DO NOT RECEIVE THE FREE REPLACEMENT BALL JOINTS WITHIN A REASONABLE TIME SHOULD CONTACT MOOG AT 1-314-385-3400.ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATION AUTO SAFETY HOTLINE AT 1-888-DASH-2-DOT (1-888-327-4236).'
,'If the component detaches during deployment, the curtain air bag may not inflate properly during a crash, increasing the risk of injury.  Additionally, if the detached component were to enter the vehicle occupant compartment, there could be an increased risk of occupant injury.	Hyundai will notify owners, and dealers will replace the driver and passenger side curtain air bags, free of charge.  The recall is expected to begin February 9, 2018.  Owners may contact Hyundai customer service at 1-855-671-3059.  Hyundai number for this recall is 171.	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.safercar.gov.'
,'Improper air bag deployment can increase the risk of an injury.	Dealers will replace the trim parts, free of charge.  Owner notification letters were mailed May 27, 2022.  Owners may contact MBUSA customer service at 1-800-367-6372.  MBUSAs number for this recall is 2022050006. 	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.nhtsa.gov.'
,'FORD CAMPAIGN NO H7H.4X4 TRUCKS EQUIPPED W/6 CYLINDER ENGINES.POSSIBILITYTHAT AN INCORRECT REAR DRIVESHAFT WAS INSTALLED IN PRODUCTION.IFTHISCONDITION EXISTS THE DRIVESHAFT WILL VIBRATE AND DEFLECT AT HIGH SPEEDS.THIS VIBRATION AND DEFLECTION COULD CAUSE SHAFT TOBREAK.A BROKEN DRIVESHAFTMIGHT ADVERSELY AFFECT CONTROL OF VEHICLE.(CORRECT BY INSPECTING AND REPLACEWITH CORRECT DRIVESHAFT WHERE NECESSARY.'
,'If the fuel pump fails, the vehicle may stall without warning, increasing the risk of a crash.	Ford will notify owners, and dealers will replace the fuel pump, free of charge.  The recall began on February 27, 2015.  Owners may contact Ford customer service at 1-866-436-7332.  Fords number for this recall is 14S30. 	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.safercar.gov.'
,'IN THE EVENT OF A SEPARATION, THE FALLING GLASS COULD CREATE A ROAD HAZARD OR STRIKE A TRAILING VEHICLE, AND CAUSE A CRASH.	WEBASTO, THE MANUFACTURER OF THE SUNROOFS, WILL BE PERFORMING THE SAFETY RECALL CAMPAIGN.  OWNERS WHO HAVE BEEN IDENTIFIED AS HAVING AN AFFECTED SUNROOF INSTALLED IN THEIR VEHICLE WILL RECEIVE A SAFETY RECALL NOTICE FROM WEBASTO.   THE SAFETY RECALL BEGAN ON OCTOBER 8, 2010.  IN ORDER TO DETERMINE IF YOUR VEHICLE IS AFFECTED, PLEASE REVIEW THE LIST OF VEHICLES PROVIDED TO US BY M & M AUTOMOTIVE] BY CLICKING ON THE HYPERLINK <a href=http://www-odi.nhtsa.dot.gov/acms/docservlet/Artemis/Public/Recalls/2011/V/RCORRD-11V041-7676.pdf>AFFECTED VEHICLE LIST</a> OR YOU MAY VISIT WEBASTOS WEBSITE AT <a href=http://www.sunroofcheck.com> WWW.SUNROOFCHECK.COM</a>. YOU CAN ALSO CALL WEBASTO TOLL-FREE AT 1-888-749-8632.	OWNERS MAY ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFETY ADMINISTRATIONS VEHICLE SAFETY HOTLINE AT 1-888-327-4236 (TTY 1-800-424-9153), OR GO TO <A HREF=HTTP://WWW.SAFERCAR.GOV>HTTP://WWW.SAFERCAR.GOV</A> .'
,'In the event of a fire, if the fire extinguisher does not function properly, it can increase the risk of injury.	"Volvo will notify owners, instructing them to contact Kidde for a replacement fire extinguisher, free of charge. The recall began on November 2, 2017.  Owners may contact Volvo customer service at 1-800-528-6586, or Kidde customer service toll-free at 1-855-262-3540, or online at www.kidde.com and click on ""Product Safety Recall"" for more information."	Owners may also contact the National Highway Traffic Safety Administration Vehicle Safety Hotline at 1-888-327-4236 (TTY 1-800-424-9153), or go to www.safercar.gov.'
,'THE DEALER WILL DO A TORQUE CHECK OF THE SELF-LOCKING NUTS AND, IF THE NUTS ARE FOUND TO BE LOOSE, THEY WILL BE TIGHTENED TO THE PROPER TORQUE SPECIFICATIONS, FREE OF CHARGE.	VEHICLE DESCRIPTION: PASSENGER VEHICLES.SYSTEM: STEERING; SELF LOCKING NUTS (STEERING COLUMN).CONSEQUENCES OF DEFECT: IF THE NUTS LOOSEN AND FALL OFF, THIS CAN LEAD TO LOSSOF VEHICLE CONTROL AND AN ACCIDENT.NOTE: PRIOR WARNING TO THE DRIVER SHOULD OCCUR IN THE FORM OF ABNORMAL STEERINGPLAY, CAUSED BY THE LOOSE NUTS.'
]


for ele in test_data:
  doc=nlp_ner(ele)
  spacy.displacy.render(doc, style="ent", jupyter=True)
  print('\n')
  print('\n')





/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [10]:
import spacy

# Assuming your model is stored in the current directory
model_directory = './model-best'
nlp = spacy.load(model_directory)

# Save the model to a directory for future use
nlp.to_disk('./savedmodel')

print(f'Model saved to path_to_save_model')


Model saved to path_to_save_model


In [19]:
import shutil
shutil.move('model-last', '/content/drive/MyDrive/Predii')

'/content/drive/MyDrive/Predii/model-last'